In [1]:
!pip install azureml-sdk

## Importing all required libraries

In [2]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

## Loading the configuration file and its informations

In [58]:
config_file_path = "config.json"

with open(config_file_path, "r") as f:
  data = json.load(f)

subscription_id = data["subscription_id"]
resource_group = data["ressource_group"]
workspace_name = data["workspace_name"]
region = data["region"]

In [59]:
print(subscription_id)
print(resource_group)
print(workspace_name)
print(region)

my_subscription_id
ml-model-deployment
diabetes-model-deployment1
centralindia


**Creating the resource group from Azure**

## Creating a workspace in Azure Machine Learning

In [10]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group = resource_group,
    location=region
)

print(f"Workspace {workspace_name} created!")

Deploying AppInsights with name diabetesinsightsd2d992ed.
Deployed AppInsights with name diabetesinsightsd2d992ed. Took 9.01 seconds.
Deploying KeyVault with name diabeteskeyvault80223ebe.
Deploying StorageAccount with name diabetesstorage10ec0b289.
Deployed KeyVault with name diabeteskeyvault80223ebe. Took 25.46 seconds.
Deployed StorageAccount with name diabetesstorage10ec0b289. Took 34.94 seconds.
Deploying Workspace with name diabetes-model-deployment1.
Deployed Workspace with name diabetes-model-deployment1. Took 38.88 seconds.
Workspace diabetes-model-deployment1 created!


## Registering the model in Azure Machine Learning

In [11]:
model_path = "diabetes_model.pkl"
model_name = "diabetes_model"

registered_model = Model.register(workspace=ws,
                                  model_path=model_path,
                                  model_name=model_name)

Registering model diabetes_model


## Creating a conda environment for the registered model

In [13]:
conda_env = Environment("my-conda-env")
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

## Creating an InferenceConfig

In [45]:
inference_config = InferenceConfig(entry_script="score.py", environment=conda_env)

## Specifying the deployment configuration for Azure Container Inference

In [15]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

## Deploying the model in the Azure container registery

In [46]:
service = Model.deploy(
    workspace=ws,
    name="diabetes-prediction-service",
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config
)

service.wait_for_deployment(show_output=True)

<ipython-input-46-451a9c37e160>:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-16 17:33:29+00:00 Registering the environment.
2024-02-16 17:33:30+00:00 Use the existing image.
2024-02-16 17:33:31+00:00 Submitting deployment to compute.
2024-02-16 17:33:38+00:00 Checking the status of deployment diabetes-prediction-service..
2024-02-16 17:35:03+00:00 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [47]:
scoring_uri = service.scoring_uri

In [48]:
scoring_uri

'http://feb50b5c-e473-4d68-920f-24cbb9a6f6e8.centralindia.azurecontainer.io/score'

## Cleaning up all the created resources

In [44]:
# Deleting the service
service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

In [ ]:
# Deleting the registred model

model_name = "diabetes_model"
registered_model = ws.models[model_name]
registered_model.delete()

In [ ]:
# deleting the workspace
ws.delete(delete_dependent_resources=True)

## Steps followed to deploy a ML model in Azure Cloud Platform

- Creating an Azure account with an appropriate subscription
- Creating a configuration file
- Creating a resource group from Azure Portal
- Creating a workspace
- Registering the model in container registery
- Deploying the model as an endpoint
- Once done, deleting the created resources